In [1]:
!pip install keras-tuner

## BASIC CNN WITHOUT TRANSFER LEARNING : KERAS

### Importing the libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
# Print TF version
print(tf.__version__)

2.4.0


### Loading inbuilt dataset

In [3]:
# Loading the Fashion MNIST dataset from the keras module
fashion_mnist = keras.datasets.fashion_mnist
# Fetch the training and the testing data
(train_img,train_labels),(test_img,test_labels) = fashion_mnist.load_data()

### Normalizing and reshaping the images

In [4]:
train_img,test_img = train_img/255.0,test_img/255.0
print("\nSHAPE OF 1 SAMPLE OF TRAINING DATA = {}\n".format(train_img[0].shape))
# Reshaping
train_img,test_img = train_img.reshape(len(train_img),train_img[0].shape[0],train_img[0].shape[0],1),test_img.reshape(len(test_img),test_img[0].shape[0],test_img[0].shape[0],1)
print("SHAPE OF TRAINING DATA = {}".format(train_img.shape))


SHAPE OF 1 SAMPLE OF TRAINING DATA = (28, 28)

SHAPE OF TRAINING DATA = (60000, 28, 28, 1)


### Build the CNN model
First, write a function that takes the hyperparameter as input, so, this function can be used later for hyperparameter tuning (using RandomSearch)

In [5]:
# Function
# hp : Hyperparameter
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters = hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size = hp.Choice('conv_1_kernel',values=[3,5]),
        activation = 'relu',
        input_shape = (28,28,1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
        kernel_size = hp.Choice('conv_2_kernel',values=[3,5]),
        activation = 'relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units = hp.Int('dense_1_units',min_value=32,max_value=64,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(
      optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )
  return model

Then, build a tuner that searches for the most optimal hyperparameters and chooses the best combination to form the best perfroming model

In [6]:
tuner_search = RandomSearch(
    hypermodel = build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    project_name = 'output'
)

In [7]:
# Start training
tuner_search.search(train_img,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.9101666808128357

Best val_accuracy So Far: 0.9101666808128357
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit


In [8]:
# Get the best model
model = tuner_search.get_best_models(num_models=1)[0]
model.summary()
# Train the selected model
model.fit(train_img,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)
# initial_epoch tells us which epoch to start the training from.
# Initialized initial_epoch to 3 coz the best model has already been through 3 epochs of training under tuner_search. So we won't retrain the model.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,545,690
Trainable params: 1,545,690
Non-trainable params: 0
_________________________________________________________________
Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1876 - accuracy: 0.9309 - val_loss